In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
%load_ext memory_profiler

In [ ]:
import pandas as pd
from utils.import_utils import *
from hydra import initialize, initialize_config_module, initialize_config_dir, compose
from src.feature_processing import *
from src.item_processing import  *
from src.utils.stats_utils import *
import plotly.express as px
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt

In [ ]:
with initialize(config_path='../configuration', version_base='1.1'):
    config = compose(config_name='main.yaml')

In [ ]:
features_class = ItemFeatureProcessing(config)

In [ ]:
df_item = features_class.df_item
self = features_class

In [ ]:
pivot_table, index_col = self.get_clean_pivot_table('f__numeric_response',remove_low_freq_col=True)

In [ ]:
# Plot Histogram of normalized and windsored distribution
for col in pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns:
    data = pivot_table[~pd.isnull(pivot_table[col])].copy()
    # # Remove extreme outliers by order of magnitude
    anomalies = get_outlier_by_magnitude(data[col], mode_deviation=2, )

    data['box_cox'] = get_box_cox_rescaled(data[col])

    # remove oyutliers by zeta score on normalized data
    lower_limit, upper_limit = get_outlier_z_score(data, 'box_cox', 3)
    lower_limit, upper_limit = get_outlier_iqr(data, 'box_cox')
    data1 = data[(~upper_limit)&(~lower_limit)].copy()    
    data = data[~anomalies].copy()
    data[col].hist(bins=10)
    data1[col].hist(bins=10)
    plt.title(col+ str(data.shape[0]) +'--'+ str(data1.shape[0]))
    plt.show()

In [ ]:
data['harvest_quantity'].value_counts()

In [ ]:
def z_score(x, series):
    return (x -series.mean())/ series.std()

In [ ]:

for col in pivot_table.drop(columns = ['interview__id', 'roster_level', 'responsible']).columns:
    data = pivot_table[~pd.isnull(pivot_table[col])].copy()
    # Remove extreme outliers by order of magnitude
    extreme_outliers = get_outlier_by_magnitude(data[col], mode_deviation=3)
    data = data[~extreme_outliers].copy()
    data['box_cox'] = get_box_cox_rescaled(data[col])
    # remove oyutliers by zeta score on normalized data
    lower_limit, upper_limit = get_outlier_z_score(data, 'box_cox', 2.5)
    data = data[(~lower_limit)&(~upper_limit)].copy()    
    pivot_table[col+'z_score'] = None
    mask = ((~pd.isnull(pivot_table[col])) &
            (~extreme_outliers) & 
             (~lower_limit) & 
             (~upper_limit))
    #pivot_table.loc[mask, col+'_z_score'] = (pivot_table[mask][col] - pivot_table[mask][col].mean()) / pivot_table[mask][col].std()
    pivot_table.loc[mask, col+'_z_score'] = (data['box_cox'] - data['box_cox'].mean()) / data['box_cox'].std()


In [ ]:
z_score_columns = [ col for col in pivot_table.columns if 'z_score' in col]
data = pivot_table[index_col+z_score_columns].groupby(['interview__id', 'responsible']).agg({col: 'mean' for col in z_score_columns})
data = data.reset_index()
data = data.copy()
for col in z_score_columns:
    data1 = data[~pd.isnull(data[col])].copy()
    fig = px.box(data1, x= 'responsible', y=col, title=col)
    fig.show()

In [ ]:
z_score_columns = [ col for col in pivot_table.columns if 'z_score' in col]
data = pivot_table[index_col+z_score_columns].groupby(['responsible']).agg({col: 'mean' for col in z_score_columns})
data = data.reset_index()
data = data.copy()
for col in z_score_columns:
    data1 = data[~pd.isnull(data[col])].copy()
    fig = px.bar(data1, x= 'responsible', y=col, title=col)
    fig.show()

In [ ]:
z_score_columns = [ col for col in pivot_table.columns if 'z_score' in col]
data = pivot_table[index_col+z_score_columns].groupby(['interview__id', 'responsible']).agg({col: 'mean' for col in z_score_columns})
data = data.reset_index()
data = data.copy()
data['z_score'] = data[z_score_columns].mean(1)
fig = px.box(data, x= 'responsible', y='z_score', title='z_score')
fig.show()

In [ ]:
z_score_columns = [ col for col in pivot_table.columns if 'z_score' in col]
data = pivot_table[index_col+z_score_columns].groupby(['responsible']).agg({col: 'mean' for col in z_score_columns})
data = data.reset_index()
data = data.copy()
data['z_score'] = data[z_score_columns].mean(1)
fig = px.box(data, x= 'responsible', y='z_score', title='z_score')
fig.show()